# Download Models

Download [google-bert/bert-large-uncased](https://www.modelscope.cn/models/google-bert/bert-large-uncased) to `/workspace/model-store`.

```bash
# tree -h bert-large-uncased/
[4.0K]  bert-large-uncased/
├── [ 571]  config.json
├── [  67]  configuration.json
├── [ 135]  flax_model.msgpack
├── [1.2G]  model.pt
├── [1.3G]  model.safetensors
├── [ 135]  pytorch_model.bin
├── [8.8K]  README.md
├── [ 135]  rust_model.ot
├── [ 135]  tf_model.h5
├── [  48]  tokenizer_config.json
├── [455K]  tokenizer.json
├── [226K]  vocab.txt
└── [ 135]  whole-word-masking.tar.gz
```

# Model Conversion

```bash
# ll /workspace/model-store/
drwxrwxr-x  4 cherry cherry 4.0K Apr  1 00:17 bert-large-uncased
-rw-r--r--  1 root   root   3.3M Apr  2 00:29 conversion_bs16_dy.txt
-rw-rw-r--  1 cherry cherry  384 Apr  2 23:37 generate_models.sh
-rw-rw-r--  1 cherry cherry  888 Apr  2 00:26 onnx_exporter.py
-rw-r--r--  1 cherry cherry 648M Apr  2 23:42 model_bs16.plan
-rw-r--r--  1 cherry root   1.3G Apr  2 00:28 model.onnx
```

TensorRT conversion logs:

```
[04/01/2025-16:29:13] [I] === Trace details ===
[04/01/2025-16:29:13] [I] Trace averages of 10 runs:
[04/01/2025-16:29:13] [I] Average on 10 runs - GPU latency: 82.6546 ms - Host latency: 83.9499 ms (enqueue 1.00352 ms)
[04/01/2025-16:29:13] [I] Average on 10 runs - GPU latency: 78.2296 ms - Host latency: 79.5337 ms (enqueue 1.18876 ms)
[04/01/2025-16:29:13] [I] Average on 10 runs - GPU latency: 78.595 ms - Host latency: 79.8829 ms (enqueue 1.12983 ms)
[04/01/2025-16:29:13] [I] 
[04/01/2025-16:29:13] [I] === Performance summary ===
[04/01/2025-16:29:13] [I] Throughput: 12.2092 qps
[04/01/2025-16:29:13] [I] Latency: min = 79.026 ms, max = 92.0858 ms, mean = 80.8949 ms, median = 79.9128 ms, percentile(90%) = 82.8573 ms, percentile(95%) = 91.9968 ms, percentile(99%) = 92.0858 ms
[04/01/2025-16:29:13] [I] Enqueue Time: min = 0.699951 ms, max = 1.56128 ms, mean = 1.06057 ms, median = 1.15906 ms, percentile(90%) = 1.22583 ms, percentile(95%) = 1.24524 ms, percentile(99%) = 1.56128 ms
[04/01/2025-16:29:13] [I] H2D Latency: min = 0.0119934 ms, max = 0.0308838 ms, mean = 0.0221546 ms, median = 0.0239258 ms, percentile(90%) = 0.029541 ms, percentile(95%) = 0.0299072 ms, percentile(99%) = 0.0308838 ms
[04/01/2025-16:29:13] [I] GPU Compute Time: min = 77.7206 ms, max = 90.7878 ms, mean = 79.5994 ms, median = 78.6023 ms, percentile(90%) = 81.5851 ms, percentile(95%) = 90.6855 ms, percentile(99%) = 90.7878 ms
[04/01/2025-16:29:13] [I] D2H Latency: min = 1.25928 ms, max = 1.2981 ms, mean = 1.27332 ms, median = 1.26892 ms, percentile(90%) = 1.29224 ms, percentile(95%) = 1.29507 ms, percentile(99%) = 1.2981 ms
[04/01/2025-16:29:13] [I] Total Host Walltime: 3.1943 s
[04/01/2025-16:29:13] [I] Total GPU Compute Time: 3.10438 s
[04/01/2025-16:29:13] [I] Explanations of the performance metrics are printed in the verbose logs.
[04/01/2025-16:29:13] [V] 
[04/01/2025-16:29:13] [V] === Explanations of the performance metrics ===
[04/01/2025-16:29:13] [V] Total Host Walltime: the host walltime from when the first query (after warmups) is enqueued to when the last query is completed.
[04/01/2025-16:29:13] [V] GPU Compute Time: the GPU latency to execute the kernels for a query.
[04/01/2025-16:29:13] [V] Total GPU Compute Time: the summation of the GPU Compute Time of all the queries. If this is significantly shorter than Total Host Walltime, the GPU may be under-utilized because of host-side overheads or data transfers.
[04/01/2025-16:29:13] [V] Throughput: the observed throughput computed by dividing the number of queries by the Total Host Walltime. If this is significantly lower than the reciprocal of GPU Compute Time, the GPU may be under-utilized because of host-side overheads or data transfers.
[04/01/2025-16:29:13] [V] Enqueue Time: the host latency to enqueue a query. If this is longer than GPU Compute Time, the GPU may be under-utilized.
[04/01/2025-16:29:13] [V] H2D Latency: the latency for host-to-device data transfers for input tensors of a single query.
[04/01/2025-16:29:13] [V] D2H Latency: the latency for device-to-host data transfers for output tensors of a single query.
[04/01/2025-16:29:13] [V] Latency: the summation of H2D Latency, GPU Compute Time, and D2H Latency. This is the latency to infer a single query.
```

In [ ]:
!docker run --gpus=all --rm -it -v /workspace:/workspace nvcr.io/nvidia/pytorch:24.12-py3 /bin/bash /workspace/model-store/generate_models.sh

# Start TritonServer

In [ ]:
!docker run --gpus=all --network=host --rm -it -v /workspace/:/workspace nvcr.io/nvidia/tritonserver:24.12-py3 tritonserver --model-control-mode=explicit --load-model=bert --model-repository=/workspace/model-store/tritonserver-model/trt-model

# Make Request

In [ ]:
import torch
import numpy as np
from transformers import BertModel
from transformers import BertTokenizer

import os
import random
import tritonclient.grpc as grpcclient
from tritonclient.utils import np_to_triton_dtype, triton_to_np_dtype
from tritonclient.utils import InferenceServerException

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

os.chdir("/workspace/model-store")


In [ ]:
def tokenize_text(text):
    enc = BertTokenizer.from_pretrained("./bert-large-uncased")
    encoded_text = enc(text, padding="max_length", max_length=512, truncation=True)
    return encoded_text["input_ids"], encoded_text["attention_mask"]

In [ ]:
random.seed(1234)

server_addr = "localhost:8001"
triton_client = grpcclient.InferenceServerClient(server_addr)

text_triton = """
                Create payload JSON and upload it on S3. 
                This will be used by Inference Recommender to run the load test.
              """

input_ids, attention_mask = tokenize_text(text_triton)
in0 = np.array(input_ids, dtype=np.int32).reshape(1, -1)
in1 = np.array(attention_mask, dtype=np.int32).reshape(1, -1)
print(in0.shape, in1.shape)
input_tensors = [
    grpcclient.InferInput("token_ids", in0.shape, np_to_triton_dtype(in0.dtype)).set_data_from_numpy(in0),
    grpcclient.InferInput("attn_mask", in1.shape, np_to_triton_dtype(in1.dtype)).set_data_from_numpy(in1),
]

output_tensors = []

model_name = "bert"
infer_rsp = triton_client.infer(model_name, inputs=input_tensors, outputs=output_tensors)

output0 = infer_rsp.as_numpy("output")
print(type(output0), output0.shape)
output1 = infer_rsp.as_numpy("3683")
print(type(output1), output1.shape)

# Caveat

- `tritonserver` stucked somehow when I profile its performance, and `nvidia-smi` shows GPU Util is 100%